In [32]:
import numpy as np
import librosa
import os
from glob import glob 
import shutil
import matplotlib.pyplot as plt
import soundfile as sf

1. Set working directory to the original full dataset with the original clips

In [34]:
os.chdir('C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\ESCAPE_II_ONECHANNEL_PROCESSED')

2. For our purposes we only want to work with data recorded from devices 306 and 308 (possibly some of the 307 cases) and we only want files ending in .wav
so we can take just those files and put them in an array

In [35]:
files = glob(os.path.join(os.getcwd(), '*.wav'), recursive=True)
workingSet = [f for f in files if os.path.basename(f).startswith('d306') or os.path.basename(f).startswith('d308')]
# workingSet = files

3. Move target files to a new directory to continue processing

In [38]:
print(len(workingSet))
destination = "C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs"

count=0
for file in workingSet:
    shutil.copy(file, destination)
    count+=1

print(f"Copied {count} files to {destination}")

82588
Copied 82588 files to C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs


4. For each .wav file, separate it into chunks with a duration of 1 second and save each chunk in the same directory with the prefix being the original file name and the postfix being segment_i.wav

In [60]:
%%time
os.chdir(r'C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original')
filesToProcess = glob.glob('*.wav')
filesToProcess = [os.path.join(os.getcwd(), file) for file in filesToProcess]
os.chdir("C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs")

# Function to process each file
def process_file(filename, chunk_duration=5):
    # Load .wav file using librosa
    y, sr = librosa.load(filename, sr=44100)
    
    # Calculate duration of the file
    duration = librosa.get_duration(y=y, sr=sr)
    
    # Calculate number of chunks
    num_chunks = int(np.ceil(duration / chunk_duration))
    
    # Iterate through each chunk
    for i in range(num_chunks):
        # Set the start and end time for the chunk
        start_time = i * chunk_duration
        end_time = min((i + 1) * chunk_duration, duration)
        
        # Extract audio segment for the chunk
        y_chunk = y[int(start_time * sr):int(end_time * sr)]
        
        # Create output file name
        output_file = f"{os.path.splitext(filename)[0]}_chunk{i+1}.wav"
        
        # Write chunk data to a new .wav file using soundfile
        sf.write(output_file, y_chunk, sr)
        
    print(f"Finished processing {filename}")

# Process each file in the directory
for file in filesToProcess:
    process_file(file)

Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r01p0120210823.wav
Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r01p0220210823.wav
Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r01p0320210823.wav
Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r01p0420210823.wav
Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r01p0520210823.wav
Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r02p0120210824.wav
Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r02p0220210824.wav
Finished processing C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original\d306sA1r02p0320210824.wav
Finished

In [63]:
base_dir = r"C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_original"
segment_dir = r"C:\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs"

for file in original:
    if not '_chunk' in file:
        src = os.path.join(base_dir, file)
        dst = os.path.join(segment_dir, file)
        shutil.move(src, dst)
        print(f'Moved {file} to {non_segment_dir}')

Moved d306sA1r01p0120210823.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r01p0220210823.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r01p0320210823.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r01p0420210823.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r01p0520210823.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r02p0120210824.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r02p0220210824.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r02p0320210824.wav to C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original
Moved d306sA1r02p0420210824.wav 

In [8]:
files = glob(os.path.join(os.getcwd(), './audio/*.wav'), recursive=True)
print(len(files))

10000


5. Verify the duration of all the chunks. Remove anything that contains no finite audio, anything shorter than a second still containing finite audio should be padded to reach a 1 second duration. Also, verify that there are no chunks with duration > 1 second.

In [26]:
from librosa.util import fix_length
chunks = glob('*.wav')
sr = 44100 
targetDuration = 1
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration <= 0.05:
        os.remove(c)
        print(f'Removed file {c} which had duration {duration}')
    elif duration < 1:
        padded = fix_length(y, size=targetDuration*sr)
        sf.write(c, padded, sr)
        print(f'Padded {c}, new duration = {librosa.get_duration(y=padded, sr=44100)}')

Removed file d306sA1r01p0420210823_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r01p0520210823_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0120210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0220210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0320210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0420210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r02p0520210824_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r09p0120210825_chunk61.wav which had duration 2.2675736961451248e-05
Padded d306sA1r09p0220210825_chunk61.wav, new duration = 1.0
Removed file d306sA1r09p0320210825_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r09p0420210825_chunk61.wav which had duration 2.2675736961451248e-05
Removed file d306sA1r09p0520210825_chunk61.wav wh

Check to make sure each file has duration of exactly 1 second to avoid problems later on

In [29]:
chunks = glob('*.wav')
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration != 1:
        print(f'{c} has duration {duration} and not duration 1')

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


You should now have your complete dataset stored in the directory chosen in step 3. This directory should contain .wav files of duration exactly 1 second which have been created from the original files chosen by you. The original files have since been removed from this directory as to ensure only 1 second long signals are present

In [39]:
import os
import glob
import shutil
import numpy as np
import soundfile as sf
import re

def recombine_segments_and_move_non_segments(base_directory, non_segment_dir):
    # Ensure the non-segment directory exists
    os.makedirs(non_segment_dir, exist_ok=True)

    for root, dirs, files in os.walk(base_directory):
        # Find all files that match the pattern '_segment_XXXX.wav'
        segment_files = sorted(glob.glob(os.path.join(root, '*_segment_*.wav')))
        
        # Dictionary to group files by their base name
        file_groups = {}
        for file in segment_files:
            base_name = os.path.splitext(os.path.basename(file))[0]
            base_name = '_'.join(base_name.split('_')[:-2])  # Remove the '_segment_XXXX' part
            if base_name not in file_groups:
                file_groups[base_name] = []
            file_groups[base_name].append(file)
        
        # Recombine files for each group
        for base_name, files in file_groups.items():
            # Sort files to ensure correct order
            def extract_segment_number(file):
                match = re.search(r'_segment_(\d+)\.wav$', file)
                if match:
                    return int(match.group(1))
                else:
                    raise ValueError(f"File does not match expected segment pattern: {file}")

            files.sort(key=extract_segment_number)
            
            combined_signal = []
            sr = None
            for file in files:
                signal, sr = sf.read(file)
                combined_signal.append(signal)
            
            # Concatenate all segments
            combined_signal = np.concatenate(combined_signal)
            
            # Create the output file path
            target_file = os.path.join(root, base_name + '.wav')
            
            # Save the combined signal to the output file
            sf.write(target_file, combined_signal, sr)
            print(f'Created {target_file}')
        
        # Move non-segment files to the separate directory
        for file in files:
            if not '_segment_' in file:
                src = os.path.join(root, file)
                dst = os.path.join(non_segment_dir, file)
                shutil.move(src, dst)
                print(f'Moved {file} to {non_segment_dir}')

# Usage
base_dir = r"C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs"
non_segment_directory = r"C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original"
recombine_segments_and_move_non_segments(base_dir, non_segment_directory)

Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r01p0120210823.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r01p0220210823.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r01p0320210823.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r01p0420210823.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r01p0520210823.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r02p0120210824.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r02p0220210824.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r02p0320210824.wav
Created C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs\d306sA1r02p0420210824.wav
C

In [ ]:
base_dir = r"C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs"
non_segment_dir = r"C:\\Users\Alec\Desktop\SIT_acoustic\ESCAPE_FORMAT_ONECHANNEL\_306_308_5_secs_original"

for file in files:
    if not '_segment_' in file:
        src = os.path.join(base_dir, file)
        dst = os.path.join(non_segment_dir, file)
        shutil.move(src, dst)
        print(f'Moved {file} to {non_segment_dir}')

